<a href="https://colab.research.google.com/github/peterbabulik/ETA/blob/main/DiscreteFourierTransform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here is the translation of the **Discrete Fourier Transform (DFT)** into a Quantum Algorithm (QFT).

### The Discrete Fourier Transform (DFT)
**The Math:** Transforming a vector from the time domain to the frequency domain.
$$ X_k = \frac{1}{\sqrt{N}} \sum_{n=0}^{N-1} x_n e^{2\pi i \frac{nk}{N}} $$

**The Quantum Translation:** **Quantum Fourier Transform (QFT)**
In Python (NumPy), `fft` takes $O(N \log N)$ time. In Quantum, QFT takes $O(\log^2 N)$ time. It doesn't give you the list of frequencies; it transforms the *state amplitudes* into the Fourier basis.

*   **The Logic:**
    *   Classical: Multiplies a vector by a matrix.
    *   Quantum: A sequence of **Hadamard** (superposition) and **Controlled-Phase** (rotation) gates.
    *   **Key Concept:** It changes the basis from "Computational" (0/1) to "Fourier" (Relative Phases).


### The "Python vs. Quantum" Logic
*   **Classical (NumPy `fft`):** You have a list of complex numbers (amplitudes). You do matrix multiplication to find their frequencies.
*   **Quantum (QFT):** You don't have a list of numbers; you have a **state** (superposition). The QFT changes the *basis* from "Binary Value" to "Relative Phase."
    *   **Input:** $|101\rangle$ (The number 5).
    *   **Output:** A superposition of all states, where the *phases* of those states encode the frequency "5".

### The Algorithm Structure
The QFT is built using two primitives:
1.  **Hadamard (H):** Creates the superposition (splits the beam).
2.  **Controlled-Phase (CP):** Adds a rotation angle. The angle gets smaller ($\pi/2, \pi/4, \pi/8$) as the distance between qubits increases. This represents the binary fraction $0.j_1 j_2 j_3...$.

### The Qiskit Code
To prove this works to a programmer, we will perform a **"Round Trip Unit Test"**:
1.  Encode the number **5** ($|101\rangle$).
2.  Apply **QFT** (Transform to frequency space).
3.  Apply **Inverse QFT** (Transform back to value space).
4.  Measure. (We should get **5** with 100% probability).




In [1]:
!pip install qiskit qiskit-aer -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.1 MB/s eta 0:00:00


In [2]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

def build_qft(n_qubits):
    """
    Constructs a QFT circuit for n qubits.
    Complexity: O(n^2) gates (much better than O(2^n) classical matrix multiplication).
    """
    qc = QuantumCircuit(n_qubits)

    # We iterate through the qubits
    # For each qubit, we apply H, then controlled rotations from all other qubits
    for i in range(n_qubits):

        # 1. Apply Hadamard to the target qubit
        # This creates the superposition step for this bit
        qc.h(i)

        # 2. Apply Controlled Phase Rotations (CP)
        # The angle depends on the distance: pi/2, pi/4, pi/8...
        # This encodes the "frequency" into the relative phase.
        for j in range(i + 1, n_qubits):
            # Calculate angle: pi / 2^(power)
            # Example: neighbor = pi/2, next neighbor = pi/4
            power = j - i + 1
            angle = np.pi / (2**power)

            # CP gate: Rotation on qubit 'j', controlled by qubit 'i'
            qc.cp(angle, j, i)

    # 3. Swap Qubits (The QFT output comes out in reverse bit order)
    # We need to mirror the register to read it correctly.
    for i in range(n_qubits // 2):
        qc.swap(i, n_qubits - i - 1)

    return qc

# --- THE "UNIT TEST" ---

# 1. Setup
n = 3
qc = QuantumCircuit(n, n)

# 2. Encode the Input Data (The number 5 = '101' binary)
# In Qiskit, q0 is LSB, so '101' means q0=1, q1=0, q2=1
qc.x(0) # 1
# q1 stays 0
qc.x(2) # 1
qc.barrier()

# 3. Apply QFT (Transform to Frequency Basis)
# If we measured here, we would get random 0s and 1s,
# because the information is hidden in the PHASES (complex numbers).
qft_circuit = build_qft(n)
qc.compose(qft_circuit, inplace=True)
qc.barrier()

# 4. Apply Inverse QFT (Transform back to Computational Basis)
# To prove QFT worked, we reverse it. If QFT is correct, Inverse(QFT(5)) == 5.
inv_qft_circuit = qft_circuit.inverse()
qc.compose(inv_qft_circuit, inplace=True)
qc.barrier()

# 5. Measure
qc.measure(range(n), range(n))

# 6. Execute
simulator = AerSimulator()
compiled_circuit = transpile(qc, simulator)
job = simulator.run(compiled_circuit, shots=1000)
result = job.result()
counts = result.get_counts()

# --- Output Results ---
print(f"--- QFT Round Trip Test ---")
print(f"Input: Number 5 (Binary 101)")
print(f"Operation: Encode -> QFT -> Inverse_QFT -> Measure")
print(f"Expected Output: {{'101': 1000}}")
print(f"Actual Output:   {counts}")

# Draw the QFT part specifically to see the recursive structure
print("\n--- The QFT Circuit Algorithm ---")
print(qft_circuit.draw(output='text'))


--- QFT Round Trip Test ---
Input: Number 5 (Binary 101)
Operation: Encode -> QFT -> Inverse_QFT -> Measure
Expected Output: {'101': 1000}
Actual Output:   {'101': 1000}

--- The QFT Circuit Algorithm ---
     ┌───┐                                        
q_0: ┤ H ├─■────────■───────────────────────────X─
     └───┘ │P(π/4)  │       ┌───┐               │ 
q_1: ──────■────────┼───────┤ H ├─■─────────────┼─
                    │P(π/8) └───┘ │P(π/4) ┌───┐ │ 
q_2: ───────────────■─────────────■───────┤ H ├─X─
                                          └───┘   


### How to Read the Circuit Diagram
When you run the code, look at the text drawing of `qft_circuit`:

1.  **H**: The qubit enters superposition.
2.  **P (Phase)**: Vertical lines connecting qubits. This is the "Feedback".
    *   Qubit 0 rotates based on what Qubit 1 is doing ($\pi/2$).
    *   Qubit 0 rotates *a little less* based on Qubit 2 ($\pi/4$).
3.  **Recursion**: This pattern repeats for Qubit 1, then Qubit 2.

### Why is this efficient?
*   **Classical FFT:** To transform 1000 inputs, you need roughly $1000 \times \log(1000) \approx 10,000$ operations.
*   **Quantum:** To transform $2^{1000}$ inputs (a number larger than atoms in the universe), you only need $1000^2 = 1,000,000$ operations.
*   **The Catch:** You can't read the result directly (the amplitudes). You have to use this as a subroutine in other algorithms (like Shor's algorithm for breaking encryption).